# Count Corona occurency
For another Story

In [43]:
import pandas as pd
import json
from pathlib import Path
import utils
import requests

In [36]:
df_votum_raw = pd.concat([
    pd.read_csv(Path('../export/votum/votum_0.csv')),
    pd.read_csv(Path('../export/votum/votum_1.csv'))
])

# Remove non members (mostly former members who are now in the Regierungsrat)
#df_votum_raw = df_votum_raw[df_votum_raw.ismember == True]

# Typecast
df_votum_raw['sitzung_date'] = pd.to_datetime(df_votum_raw['sitzung_date'])

# Remove empty texts
df_votum_raw = df_votum_raw[df_votum_raw.text.notna()]

# Calculate Amtsjahr (Amtsjahr 2020 = 1.5.2020 - 30.4.2021)
df_votum_raw['year'] = df_votum_raw['sitzung_date'].dt.year
df_votum_raw['month'] = df_votum_raw['sitzung_date'].dt.month
df_votum_raw['session'] = df_votum_raw.apply(lambda row: row['year'] if row['month'] > 5 else row['year'] - 1, axis=1)

with open(Path('../export/mitglieder.json'), encoding='utf-8') as f:
    kantonsrat = json.load(f)

# Typecast
utils.kantonsrat_to_datetime(kantonsrat)

In [38]:
df = df_votum_raw[df_votum_raw.sitzung_date >= '2019-01-01'].copy()

# Lowercase
df['text'] = df['text'].str.lower()

df['containscorona'] = df['text'].apply(lambda x: ('corona' in x) or ('covid' in x))

# How many days with corona?
df_grouped = df[df.containscorona == True].groupby('sitzung_name').agg(count = ('name', 'count'))
print("Anzahl Sitzungen mit Corona-Themen: %s" % len(df_grouped))

# Count Corona
df['count_corona'] = df['text'].apply(lambda x: x.count('corona'))
df['count_covid'] = df['text'].apply(lambda x: x.count('covid'))
df['count_sum'] = df['count_corona'] + df['count_covid']

print("Anzahl 'Corona' oder 'Covid': %s" % df['count_sum'].sum())
print("Stand: %s" % df['sitzung_date'].max())

Anzahl Sitzungen mit Corona-Themen: 108
Anzahl 'Corona' oder 'Covid': 2231
Stand: 2022-02-07 00:00:00
